![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# 2. Text Preprocessing with Spark NLP

**Note** Read this article if you want to understand the basic concepts in Spark NLP.

https://towardsdatascience.com/introduction-to-spark-nlp-foundations-and-basic-components-part-i-c83b7629ed59

## 1. Annotators and Transformer Concepts

In Spark NLP, all Annotators are either Estimators or Transformers as we see in Spark ML. An Estimator in Spark ML is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model. A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer that transforms a DataFrame with features into a DataFrame with predictions.
In Spark NLP, there are two types of annotators: AnnotatorApproach and AnnotatorModel
AnnotatorApproach extends Estimators from Spark ML, which are meant to be trained through fit(), and AnnotatorModel extends Transformers which are meant to transform data frames through transform().
Some of Spark NLP annotators have a Model suffix and some do not. The model suffix is explicitly stated when the annotator is the result of a training process. Some annotators, such as Tokenizer are transformers but do not contain the suffix Model since they are not trained, annotators. Model annotators have a pre-trained() on its static object, to retrieve the public pre-trained version of a model.
Long story short, if it trains on a DataFrame and produces a model, it’s an AnnotatorApproach; and if it transforms one DataFrame into another DataFrame through some models, it’s an AnnotatorModel (e.g. WordEmbeddingsModel) and it doesn’t take Model suffix if it doesn’t rely on a pre-trained annotator while transforming a DataFrame (e.g. Tokenizer).

By convention, there are three possible names:

Approach — Trainable annotator

Model — Trained annotator

nothing — Either a non-trainable annotator with pre-processing step or shorthand for a model

So for example, Stemmer doesn’t say Approach nor Model, however, it is a Model. On the other hand, Tokenizer doesn’t say Approach nor Model, but it has a TokenizerModel(). Because it is not “training” anything, but it is doing some preprocessing before converting into a Model. When in doubt, please refer to official documentation and API reference. Even though we will do many hands-on practices in the following articles, let us give you a glimpse to let you understand the difference between AnnotatorApproach and AnnotatorModel. As stated above, Tokenizer is an AnnotatorModel. So we need to call fit() and then transform().

Now let’s see how this can be done in Spark NLP using Annotators and Transformers. Assume that we have the following steps that need to be applied one by one on a data frame.

Split text into sentences
Tokenize
Normalize
Get word embeddings
image.png

What’s actually happening under the hood?

When we fit() on the pipeline with Spark data frame (df), its text column is fed into DocumentAssembler() transformer at first and then a new column “document” is created in Document type (AnnotatorType). As we mentioned before, this transformer is basically the initial entry point to Spark NLP for any Spark data frame. Then its document column is fed into SentenceDetector() (AnnotatorApproach) and the text is split into an array of sentences and a new column “sentences” in Document type is created. Then “sentences” column is fed into Tokenizer() (AnnotatorModel) and each sentence is tokenized and a new column “token” in Token type is created. And so on.

In [6]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())

print("Apache Spark version:", spark.version)

Spark NLP version 2.5.2
Apache Spark version: 2.4.5

## Create Spark Dataframe

In [8]:
text = 'Peter Parker is a nice guy and lives in New York'

spark_df = spark.createDataFrame([[text]]).toDF("text")

spark_df.show(truncate=False)

+------------------------------------------------+
text |
+------------------------------------------------+
Peter Parker is a nice guy and lives in New York|
+------------------------------------------------+

In [9]:
# if you want to create a spark datafarme from a list of strings
from pyspark.sql.types import StringType
text_list = ['Peter Parker is a nice guy and lives in New York.', 'Bruce Wayne is also a nice guy and lives in Gotham City.']

spark.createDataFrame(text_list, StringType()).toDF("text").show(truncate=80)

+--------------------------------------------------------+
 text|
+--------------------------------------------------------+
 Peter Parker is a nice guy and lives in New York.|
Bruce Wayne is also a nice guy and lives in Gotham City.|
+--------------------------------------------------------+

In [10]:
from pyspark.sql import Row

spark.createDataFrame(list(map(lambda x: Row(text=x), text_list))).show(truncate=80)

+--------------------------------------------------------+
 text|
+--------------------------------------------------------+
 Peter Parker is a nice guy and lives in New York.|
Bruce Wayne is also a nice guy and lives in Gotham City.|
+--------------------------------------------------------+

In [11]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/annotation/english/spark-nlp-basics/sample-sentences-en.txt

In [12]:
with open('sample-sentences-en.txt') as f:
  print (f.read())

Peter is a very good person.
My life in Russia is very interesting.
John and Peter are brothers. However they don't support each other that much.
Lucas Nogal Dunbercker is no longer happy. He has a good car though.
Europe is very culture rich. There are huge churches! and big houses!

In [13]:
spark_df = spark.read.text('sample-sentences-en.txt').toDF('text')

spark_df.show(truncate=False)

+-----------------------------------------------------------------------------+
text |
+-----------------------------------------------------------------------------+
Peter is a very good person. |
My life in Russia is very interesting. |
John and Peter are brothers. However they don't support each other that much.|
Lucas Nogal Dunbercker is no longer happy. He has a good car though. |
Europe is very culture rich. There are huge churches! and big houses! |
+-----------------------------------------------------------------------------+

In [14]:
spark_df.select('text').show(truncate=False)

+-----------------------------------------------------------------------------+
text |
+-----------------------------------------------------------------------------+
Peter is a very good person. |
My life in Russia is very interesting. |
John and Peter are brothers. However they don't support each other that much.|
Lucas Nogal Dunbercker is no longer happy. He has a good car though. |
Europe is very culture rich. There are huge churches! and big houses! |
+-----------------------------------------------------------------------------+

In [15]:
textFiles = spark.sparkContext.wholeTextFiles("./*.txt",4)
    
spark_df = textFiles.toDF(schema=['path','text'])

spark_df.show(truncate=30)

+-----------------------------+------------------------------+
 path| text|
+-----------------------------+------------------------------+
dbfs:/sample-sentences-en.txt|Peter is a very good person...|
+-----------------------------+------------------------------+

In [16]:
spark_df.select('text').take(1)

Out[10]: [Row(text="Peter is a very good person.\nMy life in Russia is very interesting.\nJohn and Peter are brothers. However they don't support each other that much.\nLucas Nogal Dunbercker is no longer happy. He has a good car though.\nEurope is very culture rich. There are huge churches! and big houses!")]

### Transformers

what are we going to do if our DataFrame doesn’t have columns in those type? Here comes transformers. In Spark NLP, we have five different transformers that are mainly used for getting the data in or transform the data from one AnnotatorType to another. Here is the list of transformers:

`DocumentAssembler`: To get through the NLP process, we need to get raw data annotated. This is a special transformer that does this for us; it creates the first annotation of type Document which may be used by annotators down the road.

`TokenAssembler`: This transformer reconstructs a Document type annotation from tokens, usually after these have been normalized, lemmatized, normalized, spell checked, etc, to use this document annotation in further annotators.

`Doc2Chunk`: Converts DOCUMENT type annotations into CHUNK type with the contents of a chunkCol.

`Chunk2Doc` : Converts a CHUNK type column back into DOCUMENT. Useful when trying to re-tokenize or do further analysis on a CHUNK result.

`Finisher`: Once we have our NLP pipeline ready to go, we might want to use our annotation results somewhere else where it is easy to use. The Finisher outputs annotation(s) values into a string.

each annotator accepts certain types of columns and outputs new columns in another type (we call this AnnotatorType).

In Spark NLP, we have the following types: 

`Document`, `token`, `chunk`, `pos`, `word_embeddings`, `date`, `entity`, `sentiment`, `named_entity`, `dependency`, `labeled_dependency`. 

That is, the DataFrame you have needs to have a column from one of these types if that column will be fed into an annotator; otherwise, you’d need to use one of the Spark NLP transformers.

## 2. Document Assembler

In Spark NLP, we have five different transformers that are mainly used for getting the data in or transform the data from one AnnotatorType to another.

That is, the DataFrame you have needs to have a column from one of these types if that column will be fed into an annotator; otherwise, you’d need to use one of the Spark NLP transformers. Here is the list of transformers: DocumentAssembler, TokenAssembler, Doc2Chunk, Chunk2Doc, and the Finisher.

So, let’s start with DocumentAssembler(), an entry point to Spark NLP annotators.

To get through the process in Spark NLP, we need to get raw data transformed into Document type at first. 

DocumentAssembler() is a special transformer that does this for us; it creates the first annotation of type Document which may be used by annotators down the road.

DocumentAssembler() comes from sparknlp.base class and has the following settable parameters. See the full list here and the source code here.

`setInputCol()` -> the name of the column that will be converted. We can specify only one column here. It can read either a String column or an Array[String]

`setOutputCol()` -> optional : the name of the column in Document type that is generated. We can specify only one column here. Default is ‘document’

`setIdCol()` -> optional: String type column with id information

`setMetadataCol()` -> optional: Map type column with metadata information

`setCleanupMode()` -> optional: Cleaning up options, 

possible values:
```
disabled: Source kept as original. This is a default.
inplace: removes new lines and tabs.
inplace_full: removes new lines and tabs but also those which were converted to strings (i.e. \n)
shrink: removes new lines and tabs, plus merging multiple spaces and blank lines to a single space.
shrink_full: remove new lines and tabs, including stringified values, plus shrinking spaces and blank lines.
```

In [24]:
from sparknlp.base import *

documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")\
.setCleanupMode("shrink")

doc_df = documentAssembler.transform(spark_df)

doc_df.show(truncate=30)

+-----------------------------+------------------------------+------------------------------+
 path| text| document|
+-----------------------------+------------------------------+------------------------------+
dbfs:/sample-sentences-en.txt|Peter is a very good person...|[[document, 0, 283, Peter i...|
+-----------------------------+------------------------------+------------------------------+

At first, we define DocumentAssembler with desired parameters and then transform the data frame with it. The most important point to pay attention to here is that you need to use a String or String[Array] type column in .setInputCol(). So it doesn’t have to be named as text. You just use the column name as it is.

In [26]:
doc_df.printSchema()

root
-- path: string (nullable = true)
-- text: string (nullable = true)
-- document: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

In [27]:
doc_df.select('document.result','document.begin','document.end').show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-----+
result |begin|end |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-----+
[Peter is a very good person. My life in Russia is very interesting. John and Peter are brothers. However they don't support each other that much. Lucas Nogal Dunbercker is no longer happy. He has a good car though. Europe is very culture rich. There are huge churches! and big houses!]|[0] |[283]|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-----+

The new column is in an array of struct type and has the parameters shown above. The annotators and transformers all come with universal metadata that would be filled down the road depending on the annotators being used. Unless you want to append other Spark NLP annotators to DocumentAssembler(), you don’t need to know what all these parameters mean for now. So we will talk about them in the following articles. You can access all these parameters with {column name}.{parameter name}.

Let’s print out the first item’s result.

In [30]:
doc_df.select("document.result").take(1)

Out[14]: [Row(result=["Peter is a very good person. My life in Russia is very interesting. John and Peter are brothers. However they don't support each other that much. Lucas Nogal Dunbercker is no longer happy. He has a good car though. Europe is very culture rich. There are huge churches! and big houses!"])]

If we would like to flatten the document column, we can do as follows.

In [32]:
import pyspark.sql.functions as F

doc_df.withColumn(
    "tmp", 
    F.explode("document"))\
    .select("tmp.*")\
    .show(truncate=False)

+-------------+-----+---+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+----------+
annotatorType|begin|end|result |metadata |embeddings|
+-------------+-----+---+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+----------+
document |0 |283|Peter is a very good person. My life in Russia is very interesting. John and Peter are brothers. However they don't support each other that much. Lucas Nogal Dunbercker is no longer happy. He has a good car though. Europe is very culture rich. There are huge churches! and big houses!|[sentence -> 0]|[] |
+-------------+-----+---+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+----------+

## 3. Sentence Detector

Finds sentence bounds in raw text.

`setCustomBounds(string)`: Custom sentence separator text

`setUseCustomOnly(bool)`: Use only custom bounds without considering those of Pragmatic Segmenter. Defaults to false. Needs customBounds.

`setUseAbbreviations(bool)`: Whether to consider abbreviation strategies for better accuracy but slower performance. Defaults to true.

`setExplodeSentences(bool)`: Whether to split sentences into different Dataset rows. Useful for higher parallelism in fat rows. Defaults to false.

In [36]:
from sparknlp.annotator import *

# we feed the document column coming from Document Assembler

sentenceDetector = SentenceDetector().\
setInputCols(['document']).\
setOutputCol('sentences')


In [37]:
sent_df = sentenceDetector.transform(doc_df)

sent_df.show(truncate=False)

+-----------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
path |text |document |sentences |
+-----------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
dbfs:/sample-sentences-en.txt|Peter is a very good person.
My life in Russia is very interesting.
John and Peter are brothers. However they don't support each other that much.
Lucas Nogal Dunbercker is no longer happy. He has a good car though.
Europe is very culture rich. There are huge churches! and big houses!|[[document, 0, 283, Peter is a very good person. My life in Russia is very interesting. John and Peter are brothers. However they don't support each other that much. Lucas Nogal Dunbercker is no longer happy. He has a good car though. Europe is very culture rich. There are huge churches! and big houses!, [sentence -> 0], []]]|[[document, 0, 27, Peter is a very good person., [sentence -> 0], []], [document, 29, 66, My life in Russia is very interesting., [sentence -> 1], []], [document, 68, 95, John and Peter are brothers., [sentence -> 2], []], [document, 97, 144, However they don't support each other that much., [sentence -> 3], []], [document, 146, 187, Lucas Nogal Dunbercker is no longer happy., [sentence -> 4], []], [document, 189, 213, He has a good car though., [sentence -> 5], []], [document, 215, 242, Europe is very culture rich., [sentence -> 6], []], [document, 244, 267, There are huge churches!, [sentence -> 7], []], [document, 269, 283, and big houses!, [sentence -> 8], []]]|
+---------

In [38]:
sent_df.select('sentences').take(3)

Out[18]: [Row(sentences=[Row(annotatorType='document', begin=0, end=27, result='Peter is a very good person.', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='document', begin=29, end=66, result='My life in Russia is very interesting.', metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='document', begin=68, end=95, result='John and Peter are brothers.', metadata={'sentence': '2'}, embeddings=[]), Row(annotatorType='document', begin=97, end=144, result="However they don't support each other that much.", metadata={'sentence': '3'}, embeddings=[]), Row(annotatorType='document', begin=146, end=187, result='Lucas Nogal Dunbercker is no longer happy.', metadata={'sentence': '4'}, embeddings=[]), Row(annotatorType='document', begin=189, end=213, result='He has a good car though.', metadata={'sentence': '5'}, embeddings=[]), Row(annotatorType='document', begin=215, end=242, result='Europe is very culture rich.', metadata={'sentence': '6'}, embeddings=[]), Row(annotatorType='document', begin=244, end=267, result='There are huge churches!', metadata={'sentence': '7'}, embeddings=[]), Row(annotatorType='document', begin=269, end=283, result='and big houses!', metadata={'sentence': '8'}, embeddings=[])])]

In [39]:
text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'
text


Out[19]: 'The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

In [40]:
spark_df = spark.createDataFrame([[text]]).toDF("text")

spark_df.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
text |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [41]:
spark_df.show(truncate=50)

+--------------------------------------------------+
 text|
+--------------------------------------------------+
The patient was prescribed 1 capsule of Advil f...|
+--------------------------------------------------+

In [42]:
doc_df = documentAssembler.transform(spark_df)

sent_df = sentenceDetector.transform(doc_df)

sent_df.show(truncate=True)

+--------------------+--------------------+--------------------+
 text| document| sentences|
+--------------------+--------------------+--------------------+
The patient was p...|[[document, 0, 33...|[[document, 0, 57...|
+--------------------+--------------------+--------------------+

In [43]:
sent_df.select('sentences.result').take(1)

Out[23]: [Row(result=['The patient was prescribed 1 capsule of Advil for 5 days .', 'He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day .', 'It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'])]

In [44]:
sentenceDetector.setExplodeSentences(True)


Out[24]: SentenceDetector_69d4eade81f8

In [45]:
sent_df = sentenceDetector.transform(doc_df)

sent_df.show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
 text| document| sentences|
+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
The patient was prescribed 1 capsule of Advil f...|[[document, 0, 339, The patient was prescribed ...|[[document, 0, 57, The patient was prescribed 1...|
The patient was prescribed 1 capsule of Advil f...|[[document, 0, 339, The patient was prescribed ...|[[document, 59, 244, He was seen by the endocri...|
The patient was prescribed 1 capsule of Advil f...|[[document, 0, 339, The patient was prescribed ...|[[document, 246, 339, It was determined that al...|
+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+

In [46]:
sent_df.select('sentences.result').show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
result |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[The patient was prescribed 1 capsule of Advil for 5 days .] |
[He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day .]|
[It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .] |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [47]:
from pyspark.sql import functions as F

sent_df.select(F.explode('sentences.result')).show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
col |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
The patient was prescribed 1 capsule of Advil for 5 days . |
He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day .|
It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months . |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

## Tokenizer

Identifies tokens with tokenization open standards. It is an **Annotator Approach, so it requires .fit()**.

A few rules will help customizing it if defaults do not fit user needs.

setExceptions(StringArray): List of tokens to not alter at all. Allows composite tokens like two worded tokens that the user may not want to split.

`addException(String)`: Add a single exception

`setExceptionsPath(String)`: Path to txt file with list of token exceptions

`caseSensitiveExceptions(bool)`: Whether to follow case sensitiveness for matching exceptions in text

`contextChars(StringArray)`: List of 1 character string to rip off from tokens, such as parenthesis or question marks. Ignored if using prefix, infix or suffix patterns.

`splitChars(StringArray)`: List of 1 character string to split tokens inside, such as hyphens. Ignored if using infix, prefix or suffix patterns.

`splitPattern (String)`: pattern to separate from the inside of tokens. takes priority over splitChars.
setTargetPattern: Basic regex rule to identify a candidate for tokenization. Defaults to \\S+ which means anything not a space

`setSuffixPattern`: Regex to identify subtokens that are in the end of the token. Regex has to end with \\z and must contain groups (). Each group will become a separate token within the prefix. Defaults to non-letter characters. e.g. quotes or parenthesis

`setPrefixPattern`: Regex to identify subtokens that come in the beginning of the token. Regex has to start with \\A and must contain groups (). Each group will become a separate token within the prefix. Defaults to non-letter characters. e.g. quotes or parenthesis

`addInfixPattern`: Add an extension pattern regex with groups to the top of the rules (will target first, from more specific to the more general).

`minLength`: Set the minimum allowed legth for each token

`maxLength`: Set the maximum allowed legth for each token

In [50]:
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

In [51]:
text = 'Peter Parker (Spiderman) is a nice guy and lives in New York but has no e-mail!'

spark_df = spark.createDataFrame([[text]]).toDF("text")


In [52]:
doc_df = documentAssembler.transform(spark_df)

token_df = tokenizer.fit(doc_df).transform(doc_df)

token_df.show(truncate=100)

+-------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
 text| document| token|
+-------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
Peter Parker (Spiderman) is a nice guy and lives in New York but has no e-mail!|[[document, 0, 78, Peter Parker (Spiderman) is a nice guy and lives in New York but has no e-mail...|[[token, 0, 4, Peter, [sentence -> 0], []], [token, 6, 11, Parker, [sentence -> 0], []], [token, ...|
+-------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+

In [53]:
token_df.select('token.result').take(1)

Out[31]: [Row(result=['Peter', 'Parker', '(', 'Spiderman', ')', 'is', 'a', 'nice', 'guy', 'and', 'lives', 'in', 'New', 'York', 'but', 'has', 'no', 'e-mail', '!'])]

In [54]:
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token") \
    .setSplitChars(['-']) \
    .setContextChars(['(', ')', '?', '!']) \
    .addException("New York") \


In [55]:
token_df = tokenizer.fit(doc_df).transform(doc_df)

token_df.select('token.result').take(1)

Out[33]: [Row(result=['Peter', 'Parker', '(', 'Spiderman', ')', 'is', 'a', 'nice', 'guy', 'and', 'lives', 'in', 'New York', 'but', 'has', 'no', 'e', 'mail', '!'])]

## Stacking Spark NLP Annotators in Spark ML Pipeline

Spark NLP provides an easy API to integrate with Spark ML Pipelines and all the Spark NLP annotators and transformers can be used within Spark ML Pipelines. So, it’s better to explain Pipeline concept through Spark ML official documentation.

What is a Pipeline anyway? In machine learning, it is common to run a sequence of algorithms to process and learn from data. 

Apache Spark ML represents such a workflow as a Pipeline, which consists of a sequence of PipelineStages (Transformers and Estimators) to be run in a specific order.

In simple terms, a pipeline chains multiple Transformers and Estimators together to specify an ML workflow. We use Pipeline to chain multiple Transformers and Estimators together to specify our machine learning workflow.

The figure below is for the training time usage of a Pipeline.

<img src="https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/databricks_notebooks/images/pipeline.png" style="float: left;">

A Pipeline is specified as a sequence of stages, and each stage is either a Transformer or an Estimator. These stages are run in order, and the input DataFrame is transformed as it passes through each stage. That is, the data are passed through the fitted pipeline in order. Each stage’s transform() method updates the dataset and passes it to the next stage. With the help of Pipelines, we can ensure that training and test data go through identical feature processing steps.

Now let’s see how this can be done in Spark NLP using Annotators and Transformers. Assume that we have the following steps that need to be applied one by one on a data frame.

- Split text into sentences
- Tokenize

And here is how we code this pipeline up in Spark NLP.

In [60]:
from pyspark.ml import Pipeline

documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

sentenceDetector = SentenceDetector().\
setInputCols(['document']).\
setOutputCol('sentences')

tokenizer = Tokenizer() \
    .setInputCols(["sentences"]) \
    .setOutputCol("token")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 sentenceDetector,
 tokenizer
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [61]:
spark_df = spark.read.text('./sample-sentences-en.txt').toDF('text')

spark_df.show(truncate=False)

+-----------------------------------------------------------------------------+
text |
+-----------------------------------------------------------------------------+
Peter is a very good person. |
My life in Russia is very interesting. |
John and Peter are brothers. However they don't support each other that much.|
Lucas Nogal Dunbercker is no longer happy. He has a good car though. |
Europe is very culture rich. There are huge churches! and big houses! |
+-----------------------------------------------------------------------------+

In [62]:
result = pipelineModel.transform(spark_df)

In [63]:
result.show(truncate=20)

+--------------------+--------------------+--------------------+--------------------+
 text| document| sentences| token|
+--------------------+--------------------+--------------------+--------------------+
Peter is a very g...|[[document, 0, 27...|[[document, 0, 27...|[[token, 0, 4, Pe...|
My life in Russia...|[[document, 0, 37...|[[document, 0, 37...|[[token, 0, 1, My...|
John and Peter ar...|[[document, 0, 76...|[[document, 0, 27...|[[token, 0, 3, Jo...|
Lucas Nogal Dunbe...|[[document, 0, 67...|[[document, 0, 41...|[[token, 0, 4, Lu...|
Europe is very cu...|[[document, 0, 68...|[[document, 0, 27...|[[token, 0, 5, Eu...|
+--------------------+--------------------+--------------------+--------------------+

In [64]:
result.printSchema()

root
-- text: string (nullable = true)
-- document: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- sentences: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- token: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

In [65]:
result.select('sentences.result').take(3)

Out[39]: [Row(result=['Peter is a very good person.']),
 Row(result=['My life in Russia is very interesting.']),
 Row(result=['John and Peter are brothers.', "However they don't support each other that much."])]

In [66]:
result.select('token').take(3)[2]

Out[40]: Row(token=[Row(annotatorType='token', begin=0, end=3, result='John', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=5, end=7, result='and', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=9, end=13, result='Peter', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=15, end=17, result='are', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=19, end=26, result='brothers', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=27, end=27, result='.', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=29, end=35, result='However', metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=37, end=40, result='they', metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=42, end=46, result="don't", metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=48, end=54, result='support', metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=56, end=59, result='each', metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=61, end=65, result='other', metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=67, end=70, result='that', metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=72, end=75, result='much', metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=76, end=76, result='.', metadata={'sentence': '1'}, embeddings=[])])

## Normalizer

Removes all dirty characters from text following a regex pattern and transforms words based on a provided dictionary

`setCleanupPatterns(patterns)`: Regular expressions list for normalization, defaults [^A-Za-z]

`setLowercase(value)`: lowercase tokens, default false

`setSlangDictionary(path)`: txt file with delimited words to be transformed into something else

In [69]:
import string
string.punctuation

Out[41]: '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [70]:
from sparknlp.annotator import *
from sparknlp.base import *

documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")\
    .setLowercase(True)\
    .setCleanupPatterns(["[^\w\d\s]"]) # remove punctuations (keep alphanumeric chars)
    # if we don't set CleanupPatterns, it will only keep alphabet letters ([^A-Za-z])


In [71]:

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 normalizer
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [72]:
result = pipelineModel.transform(spark_df)

In [73]:
result.show(truncate=20)

+--------------------+--------------------+--------------------+--------------------+
 text| document| token| normalized|
+--------------------+--------------------+--------------------+--------------------+
Peter is a very g...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, pe...|
My life in Russia...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 0, 1, my...|
John and Peter ar...|[[document, 0, 76...|[[token, 0, 3, Jo...|[[token, 0, 3, jo...|
Lucas Nogal Dunbe...|[[document, 0, 67...|[[token, 0, 4, Lu...|[[token, 0, 4, lu...|
Europe is very cu...|[[document, 0, 68...|[[token, 0, 5, Eu...|[[token, 0, 5, eu...|
+--------------------+--------------------+--------------------+--------------------+

In [74]:
result.select('normalized.result').take(3)

Out[46]: [Row(result=['peter', 'is', 'a', 'very', 'good', 'person']),
 Row(result=['my', 'life', 'in', 'russia', 'is', 'very', 'interesting']),
 Row(result=['john', 'and', 'peter', 'are', 'brothers', 'however', 'they', 'dont', 'support', 'each', 'other', 'that', 'much'])]

In [75]:
result.select('normalized').take(3)

Out[47]: [Row(normalized=[Row(annotatorType='token', begin=0, end=4, result='peter', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=6, end=7, result='is', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=9, end=9, result='a', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=11, end=14, result='very', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=16, end=19, result='good', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=21, end=26, result='person', metadata={'sentence': '0'}, embeddings=[])]),
 Row(normalized=[Row(annotatorType='token', begin=0, end=1, result='my', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=3, end=6, result='life', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=8, end=9, result='in', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=11, end=16, result='russia', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=18, end=19, result='is', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=21, end=24, result='very', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=26, end=36, result='interesting', metadata={'sentence': '0'}, embeddings=[])]),
 Row(normalized=[Row(annotatorType='token', begin=0, end=3, result='john', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=5, end=7, result='and', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=9, end=13, result='peter', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=15, end=17, result='are', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=19, end=26, result='brothers', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=29, end=35, result='however', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=37, end=40, result='they', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=42, end=45, result='dont', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=48, end=54, result='support', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=56, end=59, result='each', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=61, end=65, result='other', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=67, end=70, result='that', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=72, end=75, result='much', metadata={'sentence': '0'}, embeddings=[])])]

## Stopwords Cleaner

This annotator excludes from a sequence of strings (e.g. the output of a Tokenizer, Normalizer, Lemmatizer, and Stemmer) and drops all the stop words from the input sequences.

Functions:

`setStopWords`: The words to be filtered out. Array[String]

`setCaseSensitive`: Whether to do a case sensitive comparison over the stop words.

In [79]:
stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("token")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)\
      #.setStopWords(["no", "without"]) (e.g. read a list of words from a txt)


In [80]:
stopwords_cleaner.getStopWords()

Out[49]: ['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'her',
 'hers',
 'herself',
 'it',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 'should',
 'now',
 "i'll",
 "you'll",
 "he'll",
 "she'll",
 "we'll",
 "they'll",
 "i'd",
 "you'd",
 "he'd",
 "she'd",
 "we'd",
 "they'd",
 "i'm",
 "you're",
 "he's",
 "she's",
 "it's",
 "we're",
 "they're",
 "i've",
 "we've",
 "you've",
 "they've",
 "isn't",
 "aren't",
 "wasn't",
 "weren't",
 "haven't",
 "hasn't",
 "hadn't",
 "don't",
 "doesn't",
 "didn't",
 "won't",
 "wouldn't",
 "shan't",
 "shouldn't",
 "mustn't",
 "can't",
 "couldn't",
 'cannot',
 'could',
 "here's",
 "how's",
 "let's",
 'ought',
 "that's",
 "there's",
 "what's",
 "when's",
 "where's",
 "who's",
 "why's",
 'would']

In [81]:
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stopwords_cleaner
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [82]:
spark_df = spark.read.text('./sample-sentences-en.txt').toDF('text')

result = pipelineModel.transform(spark_df)

result.show()

+--------------------+--------------------+--------------------+--------------------+
 text| document| token| cleanTokens|
+--------------------+--------------------+--------------------+--------------------+
Peter is a very g...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, Pe...|
My life in Russia...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 3, 6, li...|
John and Peter ar...|[[document, 0, 76...|[[token, 0, 3, Jo...|[[token, 0, 3, Jo...|
Lucas Nogal Dunbe...|[[document, 0, 67...|[[token, 0, 4, Lu...|[[token, 0, 4, Lu...|
Europe is very cu...|[[document, 0, 68...|[[token, 0, 5, Eu...|[[token, 0, 5, Eu...|
+--------------------+--------------------+--------------------+--------------------+

In [83]:
result.select('cleanTokens.result').take(1)

Out[52]: [Row(result=['Peter', 'good', 'person', '.'])]

## Token Assembler

In [85]:
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

sentenceDetector = SentenceDetector().\
    setInputCols(['document']).\
    setOutputCol('sentences')

tokenizer = Tokenizer() \
    .setInputCols(["sentences"]) \
    .setOutputCol("token")

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")\
    .setLowercase(False)\

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)\

tokenassembler = TokenAssembler()\
    .setInputCols(["sentences", "cleanTokens"]) \
    .setOutputCol("clean_text")


nlpPipeline = Pipeline(stages=[
     documentAssembler,
    sentenceDetector,
     tokenizer,
     normalizer,
     stopwords_cleaner,
     tokenassembler
 ])


empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(spark_df)

result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentences| token| normalized| cleanTokens| clean_text|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Peter is a very g...|[[document, 0, 27...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, Pe...|[[token, 0, 4, Pe...|[[document, 0, 16...|
My life in Russia...|[[document, 0, 37...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 0, 1, My...|[[token, 3, 6, li...|[[document, 0, 22...|
John and Peter ar...|[[document, 0, 76...|[[document, 0, 27...|[[token, 0, 3, Jo...|[[token, 0, 3, Jo...|[[token, 0, 3, Jo...|[[document, 0, 18...|
Lucas Nogal Dunbe...|[[document, 0, 67...|[[document, 0, 41...|[[token, 0, 4, Lu...|[[token, 0, 4, Lu...|[[token, 0, 4, Lu...|[[document, 0, 34...|
Europe is very cu...|[[document, 0, 68...|[[document, 0, 27...|[[token, 0, 5, Eu...|[[token, 0, 5, Eu...|[[token, 0, 5, Eu...|[[document, 0, 18...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [86]:
result.select('text', 'clean_text.result').take(1)

Out[54]: [Row(text='Peter is a very good person.', result=['Peter good person'])]

In [87]:
# if we use TokenAssembler().setPreservePosition(True), the original borders will be preserved (dropped & unwanted chars will be replaced by spaces)
tokenassembler.setPreservePosition(True)
result2 = pipelineModel.transform(spark_df)
result2.select('clean_text.result').take(1)

Out[55]: [Row(result=['Peter good person'])]

In [88]:
result.select('text', F.explode('clean_text.result').alias('clean_text')).show(truncate=False)

+-----------------------------------------------------------------------------+-------------------------------------+
text |clean_text |
+-----------------------------------------------------------------------------+-------------------------------------+
Peter is a very good person. |Peter good person |
My life in Russia is very interesting. |life Russia interesting |
John and Peter are brothers. However they don't support each other that much.|John Peter brothers |
John and Peter are brothers. However they don't support each other that much.|However dont support much |
Lucas Nogal Dunbercker is no longer happy. He has a good car though. |Lucas Nogal Dunbercker longer happy|
Lucas Nogal Dunbercker is no longer happy. He has a good car though. |good car though |
Europe is very culture rich. There are huge churches! and big houses! |Europe culture rich |
Europe is very culture rich. There are huge churches! and big houses! |huge churches |
Europe is very culture rich. There are huge churches! and big houses! |big houses |
+-----------------------------------------------------------------------------+-------------------------------------+

In [89]:
import pyspark.sql.functions as F

result.withColumn(
    "tmp", 
    F.explode("clean_text")) \
    .select("tmp.*").select("begin","end","result","metadata.sentence").show(truncate = False)

+-----+---+-------------------------------------+--------+
begin|end|result |sentence|
+-----+---+-------------------------------------+--------+
0 |19 |Peter good person |0 |
0 |25 |life Russia interesting |0 |
0 |20 |John Peter brothers |0 |
29 |57 |However dont support much |1 |
0 |36 |Lucas Nogal Dunbercker longer happy|0 |
43 |57 |good car though |1 |
0 |20 |Europe culture rich |0 |
29 |41 |huge churches |1 |
54 |63 |big houses |2 |
+-----+---+-------------------------------------+--------+

In [90]:
# if we hadn't used Sentence Detector, this would be what we got. (tokenizer gets document instead of sentences column)

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

tokenassembler = TokenAssembler()\
    .setInputCols(["document", "cleanTokens"]) \
    .setOutputCol("clean_text")

nlpPipeline = Pipeline(stages=[
     documentAssembler,
     tokenizer,
     normalizer,
     stopwords_cleaner,
     tokenassembler
 ])


empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(spark_df)

result.select('text', 'clean_text.result').show(truncate=False)

+-----------------------------------------------------------------------------+-----------------------------------------------------+
text |result |
+-----------------------------------------------------------------------------+-----------------------------------------------------+
Peter is a very good person. |[Peter good person] |
My life in Russia is very interesting. |[life Russia interesting] |
John and Peter are brothers. However they don't support each other that much.|[John Peter brothers However dont support much] |
Lucas Nogal Dunbercker is no longer happy. He has a good car though. |[Lucas Nogal Dunbercker longer happy good car though]|
Europe is very culture rich. There are huge churches! and big houses! |[Europe culture rich huge churches big houses] |
+-----------------------------------------------------------------------------+-----------------------------------------------------+

In [91]:

result.withColumn(
    "tmp", 
    F.explode("clean_text")) \
    .select("tmp.*").select("begin","end","result","metadata.sentence").show(truncate = False)

+-----+---+---------------------------------------------------+--------+
begin|end|result |sentence|
+-----+---+---------------------------------------------------+--------+
0 |16 |Peter good person |0 |
0 |22 |life Russia interesting |0 |
0 |44 |John Peter brothers However dont support much |0 |
0 |50 |Lucas Nogal Dunbercker longer happy good car though|0 |
0 |43 |Europe culture rich huge churches big houses |0 |
+-----+---+---------------------------------------------------+--------+

## Stemmer

Returns hard-stems out of words with the objective of retrieving the meaningful part of the word

In [94]:
stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

In [95]:
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [96]:
result = pipelineModel.transform(spark_df)

result.show()

+--------------------+--------------------+--------------------+--------------------+
 text| document| token| stem|
+--------------------+--------------------+--------------------+--------------------+
Peter is a very g...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, pe...|
My life in Russia...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 0, 1, my...|
John and Peter ar...|[[document, 0, 76...|[[token, 0, 3, Jo...|[[token, 0, 3, jo...|
Lucas Nogal Dunbe...|[[document, 0, 67...|[[token, 0, 4, Lu...|[[token, 0, 4, lu...|
Europe is very cu...|[[document, 0, 68...|[[token, 0, 5, Eu...|[[token, 0, 5, eu...|
+--------------------+--------------------+--------------------+--------------------+

In [97]:
result.select('stem.result').show(truncate=False)

+-------------------------------------------------------------------------------------------+
result |
+-------------------------------------------------------------------------------------------+
[peter, i, a, veri, good, person, .] |
[my, life, in, russia, i, veri, interest, .] |
[john, and, peter, ar, brother, ., howev, thei, don't, support, each, other, that, much, .]|
[luca, nogal, dunberck, i, no, longer, happi, ., he, ha, a, good, car, though, .] |
[europ, i, veri, cultur, rich, ., there, ar, huge, church, !, and, big, hous, !] |
+-------------------------------------------------------------------------------------------+

In [98]:
import pyspark.sql.functions as F

result_df = result.select(F.explode(F.arrays_zip('token.result', 'stem.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("stem")).toPandas()

result_df.head(10)

,token,stem
0,Peter,peter
1,is,i
2,a,a
3,very,veri
4,good,good
5,person,person
6,.,.
7,My,my
8,life,life
9,in,in


## Lemmatizer

Retrieves lemmas out of words with the objective of returning a base dictionary word

In [101]:
!wget https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt -P /FileStore/

In [102]:
lemmatizer = Lemmatizer() \
    .setInputCols(["token"]) \
    .setOutputCol("lemma") \
    .setDictionary("/FileStore/AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

In [103]:
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [104]:
result = pipelineModel.transform(spark_df)

result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| token| stem| lemma|
+--------------------+--------------------+--------------------+--------------------+--------------------+
Peter is a very g...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, pe...|[[token, 0, 4, Pe...|
My life in Russia...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 0, 1, my...|[[token, 0, 1, My...|
John and Peter ar...|[[document, 0, 76...|[[token, 0, 3, Jo...|[[token, 0, 3, jo...|[[token, 0, 3, Jo...|
Lucas Nogal Dunbe...|[[document, 0, 67...|[[token, 0, 4, Lu...|[[token, 0, 4, lu...|[[token, 0, 4, Lu...|
Europe is very cu...|[[document, 0, 68...|[[token, 0, 5, Eu...|[[token, 0, 5, eu...|[[token, 0, 5, Eu...|
+--------------------+--------------------+--------------------+--------------------+--------------------+

In [105]:
result.select('lemma.result').show(truncate=False)

+---------------------------------------------------------------------------------------------+
result |
+---------------------------------------------------------------------------------------------+
[Peter, be, a, very, good, person, .] |
[My, life, in, Russia, be, very, interest, .] |
[John, and, Peter, be, brother, ., However, they, don't, support, each, other, that, much, .]|
[Lucas, Nogal, Dunbercker, be, no, long, happy, ., He, have, a, good, car, though, .] |
[Europe, be, very, culture, rich, ., There, be, huge, church, !, and, big, house, !] |
+---------------------------------------------------------------------------------------------+

In [106]:
result_df = result.select(F.explode(F.arrays_zip('token.result', 'stem.result',  'lemma.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("stem"),
        F.expr("cols['2']").alias("lemma")).toPandas()

result_df.head(10)

,token,stem,lemma
0,Peter,peter,Peter
1,is,i,be
2,a,a,a
3,very,veri,very
4,good,good,good
5,person,person,person
6,.,.,.
7,My,my,My
8,life,life,life
9,in,in,in


## NGram Generator

NGramGenerator annotator takes as input a sequence of strings (e.g. the output of a `Tokenizer`, `Normalizer`, `Stemmer`, `Lemmatizer`, and `StopWordsCleaner`). 

The parameter n is used to determine the number of terms in each n-gram. The output will consist of a sequence of n-grams where each n-gram is represented by a space-delimited string of n consecutive words with annotatorType `CHUNK` same as the Chunker annotator.

Functions:

`setN:` number elements per n-gram (>=1)

`setEnableCumulative:` whether to calculate just the actual n-grams or all n-grams from 1 through n

`setDelimiter:` Glue character used to join the tokens

In [109]:
ngrams_cum = NGramGenerator() \
            .setInputCols(["token"]) \
            .setOutputCol("ngrams") \
            .setN(3) \
            .setEnableCumulative(True)\
            .setDelimiter("_") # Default is space
    
# .setN(3) means, take bigrams and trigrams.

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 ngrams_cum
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(spark_df)

result.select('ngrams.result').show(truncate=200)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
 result|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
 [Peter, is, a, very, good, person, ., Peter_is, is_a, a_very, very_good, good_person, person_., Peter_is_a, is_a_very, a_very_good, very_good_person, good_person_.]|
[My, life, in, Russia, is, very, interesting, ., My_life, life_in, in_Russia, Russia_is, is_very, very_interesting, interesting_., My_life_in, life_in_Russia, in_Russia_is, Russia_is_very, is_very_...|
[John, and, Peter, are, brothers, ., However, they, don't, support, each, other, that, much, ., John_and, and_Peter, Peter_are, are_brothers, brothers_., ._However, However_they, they_don't, don't_...|
[Lucas, Nogal, Dunbercker, is, no, longer, happy, ., He, has, a, good, car, though, ., Lucas_Nogal, Nogal_Dunbercker, Dunbercker_is, is_no, no_longer, longer_happy, happy_., ._He, He_has, has_a, a_...|
[Europe, is, very, culture, rich, ., There, are, huge, churches, !, and, big, houses, !, Europe_is, is_very, very_culture, culture_rich, rich_., ._There, There_are, are_huge, huge_churches, churche...|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [110]:
ngrams_nonCum = NGramGenerator() \
            .setInputCols(["token"]) \
            .setOutputCol("ngrams_v2") \
            .setN(3) \
            .setEnableCumulative(False)\
            .setDelimiter("_") # Default is space
    
ngrams_nonCum.transform(result).select('ngrams_v2.result').show(truncate=200)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
 result|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
 [Peter_is_a, is_a_very, a_very_good, very_good_person, good_person_.]|
 [My_life_in, life_in_Russia, in_Russia_is, Russia_is_very, is_very_interesting, very_interesting_.]|
[John_and_Peter, and_Peter_are, Peter_are_brothers, are_brothers_., brothers_._However, ._However_they, However_they_don't, they_don't_support, don't_support_each, support_each_other, each_other_th...|
 [Lucas_Nogal_Dunbercker, Nogal_Dunbercker_is, Dunbercker_is_no, is_no_longer, no_longer_happy, longer_happy_., happy_._He, ._He_has, He_has_a, has_a_good, a_good_car, good_car_though, car_though_.]|
[Europe_is_very, is_very_culture, very_culture_rich, culture_rich_., rich_._There, ._There_are, There_are_huge, are_huge_churches, huge_churches_!, churches_!_and, !_and_big, and_big_houses, big_ho...|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

## TextMatcher

Annotator to match entire phrases (by token) provided in a file against a Document

Functions:

setEntities(path, format, options): Provides a file with phrases to match. Default: Looks up path in configuration.

path: a path to a file that contains the entities in the specified format.

readAs: the format of the file, can be one of {ReadAs.LINE_BY_LINE, ReadAs.SPARK_DATASET}. Defaults to LINE_BY_LINE.

options: a map of additional parameters. Defaults to {“format”: “text”}.

entityValue : Value for the entity metadata field to indicate which chunk comes from which textMatcher when there are multiple textMatchers.

mergeOverlapping : whether to merge overlapping matched chunks. Defaults false

caseSensitive : whether to match regardless of case. Defaults true

In [113]:
# first method for doing this, second option below
import urllib.request
with urllib.request.urlopen('https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed-sample.csv') as f: 
    content = f.read().decode('utf-8')
    dbutils.fs.put("/dbfs/tmp/pubmed/pubmed-sample.csv", content)

Wrote 10484510 bytes.

In [114]:
%sh
TMP=/dbfs/tmp/pubmed
if [ ! -d "$TMP" ]; then
    mkdir $TMP
    cd $TMP
    wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed-sample.csv
fi

--2020-08-18 02:41:14-- https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed-sample.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.141.94
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.141.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10484510 (10.0M) [text/csv]
Saving to: ‘pubmed-sample.csv’

 0K .......... .......... .......... .......... .......... 0% 559K 18s
 50K .......... .......... .......... .......... .......... 0% 1.09M 14s
 100K .......... .......... .......... .......... .......... 1% 596K 15s
 150K .......... .......... .......... .......... .......... 1% 17.4M 11s
 200K .......... .......... .......... .......... .......... 2% 21.6M 9s
 250K .......... .......... .......... .......... .......... 2% 664K 10s
 300K .......... .......... .......... .......... .......... 3% 18.8M 9s
 350K .......... .......... .......... .......... .......... 3% 21.4M 7s
 400K .......... .......... .......... .......... .......... 4% 22.6M 7s
 450K .......... .......... .......... .......... .......... 4% 24.8M 6s
 500K .......... .......... .......... .......... .......... 5% 763K 7s
 550K .......... .......... .......... .......... .......... 5% 8.80M 6s
 600K .......... .......... .......... .......... .......... 6% 16.9M 6s
 650K .......... .......... .......... .......... .......... 6% 22.1M 5s
 700K .......... .......... .......... .......... .......... 7% 22.6M 5s
 750K .......... .......... .......... .......... .......... 7% 1.82M 5s
 800K .......... .......... .......... .......... .......... 8% 14.0M 5s
 850K .......... .......... .......... .......... .......... 8% 22.9M 4s
 900K .......... .......... .......... .......... .......... 9% 24.8M 4s
 950K .......... .......... .......... .......... .......... 9% 21.1M 4s
 1000K .......... .......... .......... .......... .......... 10% 20.5M 4s
 1050K .......... .......... .......... .......... .......... 10% 1.69M 4s
 1100K .......... .......... .......... .......... .......... 11% 9.97M 4s
 1150K .......... .......... .......... .......... .......... 11% 18.5M 3s
 1200K .......... .......... .......... .......... .......... 12% 20.2M 3s
 1250K .......... .......... .......... .......... .......... 12% 19.3M 3s
 1300K .......... .......... .......... .......... .......... 13% 25.3M 3s
 1350K .......... .......... .......... .......... .......... 13% 28.4M 3s
 1400K .......... .......... .......... .......... .......... 14% 27.6M 3s
 1450K .......... .......... .......... .......... .......... 14% 25.5M 3s
 1500K .......... .......... .......... .......... .......... 15% 24.0M 3s
 1550K .......... .......... .......... .......... .......... 15% 25.8M 3s
 1600K .......... .......... .......... .......... .......... 16% 3.05M 3s
 1650K .......... .......... .......... .......... .......... 16% 16.3M 2s
 1700K .......... .......... .......... .......... .......... 17% 23.3M 2s
 1750K .......... .......... .......... .......... .......... 17% 27.7M 2s
 1800K .......... .......... .......... .......... .......... 18% 26.6M 2s
 1850K .......... .......... .......... .......... .......... 18% 26.7M 2s
 1900K .......... .......... .......... .......... .......... 19% 24.7M 2s
 1950K .......... .......... .......... .......... .......... 19% 27.4M 2s
 2000K .......... .......... .......... .......... .......... 20% 25.4M 2s
 2050K .......... .......... .......... .......... .......... 20% 26.1M 2s
 2100K .......... .......... .......... .......... .......... 20% 22.8M 2s
 2150K .......... .......... .......... .......... .......... 21% 26.5M 2s
 2200K .......... .......... .......... .......... .......... 21% 2.49M 2s
 2250K .......... .......... .......... .......... .......... 22% 12.7M 2s
 2300K .......... .......... .......... .......... .......... 22% 17.1M 2s
 2350K .......... .......... .......... .......... .......... 23% 28.5M 2s
 2400K .......... .......... .......... .......... .......... 

In [115]:
import pyspark.sql.functions as F

pubMedDF = spark.read\
                .option("header", "true")\
                .csv("/dbfs/tmp/pubmed/pubmed-sample.csv")\
                .filter("AB IS NOT null")\
                .withColumnRenamed("AB", "text")\
                .drop("TI")

pubMedDF.show(truncate=50)

+--------------------------------------------------+
 text|
+--------------------------------------------------+
The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|
BACKGROUND: At present, it is one of the most i...|
OBJECTIVE: To investigate the relationship betw...|
Combined EEG/fMRI recording has been used to lo...|
Kohlschutter syndrome is a rare neurodegenerati...|
Statistical analysis of neuroimages is commonly...|
The synthetic DOX-LNA conjugate was characteriz...|
Our objective was to compare three different me...|
We conducted a phase II study to assess the eff...|
"Monomeric sarcosine oxidase (MSOX) is a flavoe...|
We presented the tachinid fly Exorista japonica...|
The literature dealing with the water conductin...|
A novel approach to synthesize chitosan-O-isopr...|
An HPLC-ESI-MS-MS method has been developed for...|
The localizing and lateralizing values of eye a...|
OBJECTIVE: To evaluate the effectiveness and ac...|
For the construction of new combinatorial libra...|
We report the results of a screen for genetic a...|
Intraparenchymal pericatheter cyst is rarely re...|
It is known that patients with Klinefelter's sy...|
+--------------------------------------------------+
only showing top 20 rows

In [116]:
pubMedDF.select('text').take(2)

Out[82]: [Row(text='The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK) channel family. Here we describe the genomicorganization of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene forType II diabetes mellitus in the Pima Indian population. The gene spansapproximately 7.6 kb and contains one noncoding and two coding exons separated byapproximately 2.2 and approximately 2.6 kb introns, respectively. We identified14 single nucleotide polymorphisms (SNPs), including one that predicts aVal366Ala substitution, and an 8 base-pair (bp) insertion/deletion. Ourexpression studies revealed the presence of the transcript in various humantissues including pancreas, and two major insulin-responsive tissues: fat andskeletal muscle. The characterization of the KCNJ9 gene should facilitate furtherstudies on the function of the KCNJ9 protein and allow evaluation of thepotential role of the locus in Type II diabetes.'),
 Row(text='BACKGROUND: At present, it is one of the most important issues for the treatment of breast cancer to develop the standard therapy for patients previously treated with anthracyclines and taxanes. With the objective of determining the usefulnessof vinorelbine monotherapy in patients with advanced or recurrent breast cancerafter standard therapy, we evaluated the efficacy and safety of vinorelbine inpatients previously treated with anthracyclines and taxanes. METHODS: Vinorelbinewas administered at a dose level of 25 mg/m(2) intravenously on days 1 and 8 of a3 week cycle. Patients were given three or more cycles in the absence of tumorprogression. A maximum of nine cycles were administered. RESULTS: The responserate in 50 evaluable patients was 20.0% (10 out of 50; 95% confidence interval,10.0-33.7%). Responders plus those who had minor response (MR) or no change (NC) accounted for 58.0% [10 partial responses (PRs) + one MR + 18 NCs out of 50]. TheKaplan-Meier estimate (50% point) of time to progression (TTP) was 115.0 days.The response rate in the visceral organs was 17.3% (nine PRs out of 52). Themajor toxicity was myelosuppression, which was reversible and did not requirediscontinuation of treatment. CONCLUSION: The results of this study show thatvinorelbine monotherapy is useful in patients with advanced or recurrent breastcancer previously exposed to both anthracyclines and taxanes.')]

In [117]:
%sh
TMP=/FileStore/entities
if [ ! -d "$TMP" ]; then
    mkdir $TMP
fi

In [118]:
# write the target entities to txt file 

entities = ['KCNJ9', 'GIRK', 'diabetes mellitus', 'nucleotide polymorphisms']
content = ''
for e in entities:
  content = content + "\n" + e
dbutils.fs.put("dbfs:/tmp/pubmed/clinical_entities.txt", content)


entities = ['breast cancer', 'colon cancer', 'lung cancer', 'monotherapy', 'therapy']
content=''
for e in entities:
  content = content + "\n" + e
dbutils.fs.put("dbfs:/tmp/pubmed/cancer_entities.txt", content)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-21558677108689> in <module> 
 5 for e in entities : 
 6 content = content + "\n" + e
 ----> 7 dbutils . fs . put ( "dbfs:/tmp/pubmed/clinical_entities.txt" , content ) 
 8 
 9 

 /local_disk0/tmp/1597717970634-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 312 exc . __context__ = None 
 313 exc . __cause__ = None 
 --> 314 raise exc
 315 return f_with_exception_handling
 316 

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.put.
: org.apache.hadoop.fs.FileAlreadyExistsException: /oregon-prod/3418543878422327/tmp/pubmed/clinical_entities.txt already exists
	at com.databricks.s3a.S3AFileSystem.create(S3AFileSystem.java:629)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$create$1$$anonfun$apply$10$$anonfun$apply$11.apply(DatabricksFileSystemV2.scala:544)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$create$1$$anonfun$apply$10$$anonfun$apply$11.apply(DatabricksFileSystemV2.scala:541)
	at com.databricks.s3a.S3AExeceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:119)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$create$1$$anonfun$apply$10.apply(DatabricksFileSystemV2.scala:541)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$create$1$$anonfun$apply$10.apply(DatabricksFileSystemV2.scala:541)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$withUserContextRecorded$1.apply(DatabricksFileSystemV2.scala:936)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:450)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:275)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:450)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withUserContextRecorded(DatabricksFileSystemV2.scala:909)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$create$1.apply(DatabricksFileSystemV2.scala:540)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$create$1.apply(DatabricksFileSystemV2.scala:540)
	at com.databricks.logging.UsageLogging$$anonfun$recordOperation$1.apply(UsageLogging.scala:428)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:450)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:275)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:450)
	at com.databricks.logging.UsageLogging$class.recordOperation(UsageLogging.scala:409)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:450)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.create(DatabricksFileSystemV2.scala:537)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.create(DatabricksFileSystem.scala:128)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:911)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:892)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:789

In [119]:
clinical_entity_extractor = TextMatcher() \
    .setInputCols(["document",'token'])\
    .setOutputCol("clinical_entities")\
    .setEntities("dbfs:/tmp/pubmed/clinical_entities.txt")\
    .setCaseSensitive(False)\
    .setEntityValue('clinical_entity')

cancer_entity_extractor = TextMatcher() \
    .setInputCols(["document",'token'])\
    .setOutputCol("cancer_entities")\
    .setEntities("dbfs:/tmp/entities/cancer_entities.txt")\
    .setCaseSensitive(False)\
    .setEntityValue('cancer_entity')


nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 clinical_entity_extractor,
 cancer_entity_extractor
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)


In [120]:
result = pipelineModel.transform(pubMedDF.limit(30))

In [121]:
result.select('clinical_entities.result','cancer_entities.result').take(2)

Out[108]: [Row(result=['KCNJ9', 'GIRK', 'KCNJ9', 'diabetes mellitus', 'nucleotide polymorphisms', 'KCNJ9', 'KCNJ9'], result=[]),
 Row(result=[], result=['breast cancer', 'therapy', 'monotherapy', 'therapy', 'monotherapy'])]

In [122]:
result.select('clinical_entities','cancer_entities').take(2)

Out[109]: [Row(clinical_entities=[Row(annotatorType='chunk', begin=10, end=14, result='KCNJ9', metadata={'sentence': '0', 'chunk': '0', 'entity': 'clinical_entity'}, embeddings=[]), Row(annotatorType='chunk', begin=103, end=106, result='GIRK', metadata={'sentence': '0', 'chunk': '1', 'entity': 'clinical_entity'}, embeddings=[]), Row(annotatorType='chunk', begin=173, end=177, result='KCNJ9', metadata={'sentence': '0', 'chunk': '2', 'entity': 'clinical_entity'}, embeddings=[]), Row(annotatorType='chunk', begin=238, end=254, result='diabetes mellitus', metadata={'sentence': '0', 'chunk': '3', 'entity': 'clinical_entity'}, embeddings=[]), Row(annotatorType='chunk', begin=471, end=494, result='nucleotide polymorphisms', metadata={'sentence': '0', 'chunk': '4', 'entity': 'clinical_entity'}, embeddings=[]), Row(annotatorType='chunk', begin=801, end=805, result='KCNJ9', metadata={'sentence': '0', 'chunk': '5', 'entity': 'clinical_entity'}, embeddings=[]), Row(annotatorType='chunk', begin=868, end=872, result='KCNJ9', metadata={'sentence': '0', 'chunk': '6', 'entity': 'clinical_entity'}, embeddings=[])], cancer_entities=[]),
 Row(clinical_entities=[], cancer_entities=[Row(annotatorType='chunk', begin=84, end=96, result='breast cancer', metadata={'sentence': '0', 'chunk': '0', 'entity': 'cancer_entity'}, embeddings=[]), Row(annotatorType='chunk', begin=122, end=128, result='therapy', metadata={'sentence': '0', 'chunk': '1', 'entity': 'cancer_entity'}, embeddings=[]), Row(annotatorType='chunk', begin=258, end=268, result='monotherapy', metadata={'sentence': '0', 'chunk': '2', 'entity': 'cancer_entity'}, embeddings=[]), Row(annotatorType='chunk', begin=337, end=343, result='therapy', metadata={'sentence': '0', 'chunk': '3', 'entity': 'cancer_entity'}, embeddings=[]), Row(annotatorType='chunk', begin=1279, end=1289, result='monotherapy', metadata={'sentence': '0', 'chunk': '4', 'entity': 'cancer_entity'}, embeddings=[])])]

In [123]:
result_df = result.select(F.explode(F.arrays_zip('clinical_entities.result', 'clinical_entities.begin',  'clinical_entities.end')).alias("cols")) \
.select(F.expr("cols['0']").alias("clinical_entities"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end")).toPandas()
result_df.head(10)

,clinical_entities,begin,end
0,KCNJ9,10,14
1,GIRK,103,106
2,KCNJ9,173,177
3,diabetes mellitus,238,254
4,nucleotide polymorphisms,471,494
5,KCNJ9,801,805
6,KCNJ9,868,872


In [124]:
result_df = result.select(F.explode(F.arrays_zip('cancer_entities.result', 'cancer_entities.begin',  'cancer_entities.end')).alias("cols")) \
.select(F.expr("cols['0']").alias("cancer_entities"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end")).toPandas()

result_df.head(10)

,cancer_entities,begin,end
0,therapy,545,551
1,therapy,66,72
2,therapy,510,516


## RegexMatcher

In [126]:
rules = '''
renal\s\w+, started with 'renal'
cardiac\s\w+, started with 'cardiac'
\w*ly\b, ending with 'ly'
\S*\d+\S*, match any word that contains numbers
(\d+).?(\d*)\s*(mg|ml|g), match medication metrics
'''

dbutils.fs.put("dbfs:/tmp/pubmed/regex_rules.txt", rules)

Wrote 195 bytes.
Out[115]: True

In [127]:
import os

documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

regex_matcher = RegexMatcher()\
    .setInputCols('document')\
    .setStrategy("MATCH_ALL")\
    .setOutputCol("regex_matches")\
    .setExternalRules(path='dbfs:/tmp/pubmed/regex_rules.txt', delimiter=',')
    

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 regex_matcher
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

match_df = pipelineModel.transform(pubMedDF)

match_df.select('regex_matches.result').take(3)

Out[116]: [Row(result=['inwardly', 'family', 'spansapproximately', 'byapproximately', 'approximately', 'respectively', 'poly', 'KCNJ9', '3.3,', 'GIRK3)', 'KCNJ9', '1q21-23', '7.6', '2.2', '2.6', 'identified14', 'aVal366Ala', '8', 'KCNJ9', 'KCNJ9', '9 g']),
 Row(result=['previously', 'previously', 'intravenously', 'previously', '25', 'mg/m(2)', '1', '8', 'a3', '50', '20.0%', '(10', '50;', '95%', 'interval,10.0-33.7%).', '58.0%', '[10', '18', '50].', '(50%', '115.0', '17.3%', '52).', '25 mg']),
 Row(result=['renal failure', 'cardiac surgery', 'cardiac surgery', 'cardiac surgical', 'early', 'statistically', 'analy', '1995', '2005', '=9796).', '2.9', '11years).', '11.3%', '1105),', '7.2%', '30%', '0.0001),', '1.55,95%', '1.42-1.70,', '0.0001).'])]

In [128]:
match_df.select('text','regex_matches.result')\
.toDF('text','matches').filter(F.size('matches')>1)\
.show(truncate=50)


+--------------------------------------------------+--------------------------------------------------+
 text| matches|
+--------------------------------------------------+--------------------------------------------------+
The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|[inwardly, family, spansapproximately, byapprox...|
BACKGROUND: At present, it is one of the most i...|[previously, previously, intravenously, previou...|
OBJECTIVE: To investigate the relationship betw...|[renal failure, cardiac surgery, cardiac surger...|
Combined EEG/fMRI recording has been used to lo...|[normally, significantly, effectively, analy, o...|
Statistical analysis of neuroimages is commonly...|[analy, commonly, overly, normally, thatsuccess...|
The synthetic DOX-LNA conjugate was characteriz...| [wasanaly, substantially]|
Our objective was to compare three different me...|[daily, only, Conversely, Hourly, hourly, Hourl...|
We conducted a phase II study to assess the eff...|[analy, respectively, generally, 5-fluorouracil...|
"Monomeric sarcosine oxidase (MSOX) is a flavoe...|[cataly, methylgly, gly, ethylgly, dimethylgly,...|
We presented the tachinid fly Exorista japonica...| [fly, fly, fly, fly, fly]|
The literature dealing with the water conductin...| [generally, mathematically, especially]|
A novel approach to synthesize chitosan-O-isopr...|[efficiently, poly, chitosan-O-isopropyl-5'-O-d...|
An HPLC-ESI-MS-MS method has been developed for...|[chromatographically, respectively, successfull...|
The localizing and lateralizing values of eye a...| [early, early]|
OBJECTIVE: To evaluate the effectiveness and ac...|[weekly, respectively, theanaly, 2006, 2007,, 2...|
We report the results of a screen for genetic a...|[poly, threepoly, significantly, analy, actuall...|
Intraparenchymal pericatheter cyst is rarely re...| [rarely, possibly, unusually, Early]|
PURPOSE: To compare the effectiveness, potentia...|[analy, comparatively, wassignificantly, respec...|
We have demonstrated a new type of all-optical ...|[approximately, fully, approximately, approxima...|
Physalis peruviana (PP) is a widely used medici...|[widely, (20,, 40,, 60,, 80, 95%, 100, 95%, (82...|
+--------------------------------------------------+--------------------------------------------------+
only showing top 20 rows

## Text Cleaning with UDF

In [130]:
text = '<h1 style="color: #5e9ca0;">Have a great <span  style="color: #2b2301;">birth</span> day!</h1>'

text_df = spark.createDataFrame([[text]]).toDF("text")

import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType

clean_text = lambda s: re.sub(r'<[^>]*>', '', s)

text_df.withColumn('cleaned', udf(clean_text, StringType())('text')).select('text','cleaned').show(truncate= False)



+----------------------------------------------------------------------------------------------+-----------------------+
text |cleaned |
+----------------------------------------------------------------------------------------------+-----------------------+
<h1 style="color: #5e9ca0;">Have a great <span style="color: #2b2301;">birth</span> day!</h1>|Have a great birth day!|
+----------------------------------------------------------------------------------------------+-----------------------+

In [131]:
find_not_alnum_count = lambda s: len([i for i in s if not i.isalnum() and i!=' '])

find_not_alnum_count("it's your birth day!")

In [132]:
find_not_alnum_count = lambda s: len([i for i in s if not i.isalnum() and i!=' '])

find_not_alnum_count("it's your birth day!")

Out[119]: 2

In [133]:
text = '<h1 style="color: #5e9ca0;">Have a great <span  style="color: #2b2301;">birth</span> day!</h1>'

find_not_alnum_count(text)

Out[120]: 23

In [134]:
text_df.withColumn('cleaned', udf(find_not_alnum_count, IntegerType())('text')).select('text','cleaned').show(truncate= False)

+----------------------------------------------------------------------------------------------+-------+
text |cleaned|
+----------------------------------------------------------------------------------------------+-------+
<h1 style="color: #5e9ca0;">Have a great <span style="color: #2b2301;">birth</span> day!</h1>|23 |
+----------------------------------------------------------------------------------------------+-------+

## Finisher

***Finisher:*** Once we have our NLP pipeline ready to go, we might want to use our annotation results somewhere else where it is easy to use. The Finisher outputs annotation(s) values into a string.

If we just want the desired output column in the final dataframe, we can use Finisher to drop previous stages in the final output and gte the `result` from the process.

This is very handy when you want to use the output from Spark NLP annotator as an input to another Spark ML transformer.

Settable parameters are:

`setInputCols()`

`setOutputCols()`

`setCleanAnnotations(True)` -> Whether to remove intermediate annotations

`setValueSplitSymbol(“#”)` -> split values within an annotation character

`setAnnotationSplitSymbol(“@”)` -> split values between annotations character

`setIncludeMetadata(False)` -> Whether to include metadata keys. Sometimes useful in some annotations.

`setOutputAsArray(False)` -> Whether to output as Array. Useful as input for other Spark transformers.

In [137]:
finisher = Finisher() \
    .setInputCols(["regex_matches"]) \
    .setIncludeMetadata(False) # set to False to remove metadata

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 regex_matcher,
 finisher
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

match_df = pipelineModel.transform(pubMedDF)

match_df.show(truncate = 50)

+--------------------------------------------------+--------------------------------------------------+
 text| finished_regex_matches|
+--------------------------------------------------+--------------------------------------------------+
The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|[inwardly, family, spansapproximately, byapprox...|
BACKGROUND: At present, it is one of the most i...|[previously, previously, intravenously, previou...|
OBJECTIVE: To investigate the relationship betw...|[renal failure, cardiac surgery, cardiac surger...|
Combined EEG/fMRI recording has been used to lo...|[normally, significantly, effectively, analy, o...|
Kohlschutter syndrome is a rare neurodegenerati...| [family]|
Statistical analysis of neuroimages is commonly...|[analy, commonly, overly, normally, thatsuccess...|
The synthetic DOX-LNA conjugate was characteriz...| [wasanaly, substantially]|
Our objective was to compare three different me...|[daily, only, Conversely, Hourly, hourly, Hourl...|
We conducted a phase II study to assess the eff...|[analy, respectively, generally, 5-fluorouracil...|
"Monomeric sarcosine oxidase (MSOX) is a flavoe...|[cataly, methylgly, gly, ethylgly, dimethylgly,...|
We presented the tachinid fly Exorista japonica...| [fly, fly, fly, fly, fly]|
The literature dealing with the water conductin...| [generally, mathematically, especially]|
A novel approach to synthesize chitosan-O-isopr...|[efficiently, poly, chitosan-O-isopropyl-5'-O-d...|
An HPLC-ESI-MS-MS method has been developed for...|[chromatographically, respectively, successfull...|
The localizing and lateralizing values of eye a...| [early, early]|
OBJECTIVE: To evaluate the effectiveness and ac...|[weekly, respectively, theanaly, 2006, 2007,, 2...|
For the construction of new combinatorial libra...| [newly]|
We report the results of a screen for genetic a...|[poly, threepoly, significantly, analy, actuall...|
Intraparenchymal pericatheter cyst is rarely re...| [rarely, possibly, unusually, Early]|
It is known that patients with Klinefelter's sy...| []|
+--------------------------------------------------+--------------------------------------------------+
only showing top 20 rows

In [138]:
match_df.printSchema()

root
-- text: string (nullable = true)
-- finished_regex_matches: array (nullable = true)
 |-- element: string (containsNull = true)

In [139]:
match_df.filter(F.size('finished_regex_matches')>1).show(truncate = 50)

+--------------------------------------------------+--------------------------------------------------+
 text| finished_regex_matches|
+--------------------------------------------------+--------------------------------------------------+
The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|[inwardly, family, spansapproximately, byapprox...|
BACKGROUND: At present, it is one of the most i...|[previously, previously, intravenously, previou...|
OBJECTIVE: To investigate the relationship betw...|[renal failure, cardiac surgery, cardiac surger...|
Combined EEG/fMRI recording has been used to lo...|[normally, significantly, effectively, analy, o...|
Statistical analysis of neuroimages is commonly...|[analy, commonly, overly, normally, thatsuccess...|
The synthetic DOX-LNA conjugate was characteriz...| [wasanaly, substantially]|
Our objective was to compare three different me...|[daily, only, Conversely, Hourly, hourly, Hourl...|
We conducted a phase II study to assess the eff...|[analy, respectively, generally, 5-fluorouracil...|
"Monomeric sarcosine oxidase (MSOX) is a flavoe...|[cataly, methylgly, gly, ethylgly, dimethylgly,...|
We presented the tachinid fly Exorista japonica...| [fly, fly, fly, fly, fly]|
The literature dealing with the water conductin...| [generally, mathematically, especially]|
A novel approach to synthesize chitosan-O-isopr...|[efficiently, poly, chitosan-O-isopropyl-5'-O-d...|
An HPLC-ESI-MS-MS method has been developed for...|[chromatographically, respectively, successfull...|
The localizing and lateralizing values of eye a...| [early, early]|
OBJECTIVE: To evaluate the effectiveness and ac...|[weekly, respectively, theanaly, 2006, 2007,, 2...|
We report the results of a screen for genetic a...|[poly, threepoly, significantly, analy, actuall...|
Intraparenchymal pericatheter cyst is rarely re...| [rarely, possibly, unusually, Early]|
PURPOSE: To compare the effectiveness, potentia...|[analy, comparatively, wassignificantly, respec...|
We have demonstrated a new type of all-optical ...|[approximately, fully, approximately, approxima...|
Physalis peruviana (PP) is a widely used medici...|[widely, (20,, 40,, 60,, 80, 95%, 100, 95%, (82...|
+--------------------------------------------------+--------------------------------------------------+
only showing top 20 rows

## LightPipeline

LightPipelines are Spark NLP specific Pipelines, equivalent to Spark ML Pipeline, but meant to deal with smaller amounts of data. They’re useful working with small datasets, debugging results, or when running either training or prediction from an API that serves one-off requests.

Spark NLP LightPipelines are Spark ML pipelines converted into a single machine but the multi-threaded task, becoming more than 10x times faster for smaller amounts of data (small is relative, but 50k sentences are roughly a good maximum). To use them, we simply plug in a trained (fitted) pipeline and then annotate a plain text. We don't even need to convert the input text to DataFrame in order to feed it into a pipeline that's accepting DataFrame as an input in the first place. This feature would be quite useful when it comes to getting a prediction for a few lines of text from a trained ML model.

 **It is nearly 20x faster than using Spark ML Pipeline**

`LightPipeline(someTrainedPipeline).annotate(someStringOrArray)`

In [142]:
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

pipelineModel.transform(spark_df).show()


+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| token| stem| lemma|
+--------------------+--------------------+--------------------+--------------------+--------------------+
Peter is a very g...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, pe...|[[token, 0, 4, Pe...|
My life in Russia...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 0, 1, my...|[[token, 0, 1, My...|
John and Peter ar...|[[document, 0, 76...|[[token, 0, 3, Jo...|[[token, 0, 3, jo...|[[token, 0, 3, Jo...|
Lucas Nogal Dunbe...|[[document, 0, 67...|[[token, 0, 4, Lu...|[[token, 0, 4, lu...|[[token, 0, 4, Lu...|
Europe is very cu...|[[document, 0, 68...|[[token, 0, 5, Eu...|[[token, 0, 5, eu...|[[token, 0, 5, Eu...|
+--------------------+--------------------+--------------------+--------------------+--------------------+

In [143]:
from sparknlp.base import LightPipeline

light_model = LightPipeline(pipelineModel)

light_result = light_model.annotate("John and Peter are brothers. However they don't support each other that much.")

In [144]:
light_result.keys()

Out[127]: dict_keys(['document', 'token', 'stem', 'lemma'])

In [145]:
list(zip(light_result['token'], light_result['stem'], light_result['lemma']))

Out[128]: [('John', 'john', 'John'),
 ('and', 'and', 'and'),
 ('Peter', 'peter', 'Peter'),
 ('are', 'ar', 'be'),
 ('brothers', 'brother', 'brother'),
 ('.', '.', '.'),
 ('However', 'howev', 'However'),
 ('they', 'thei', 'they'),
 ("don't", "don't", "don't"),
 ('support', 'support', 'support'),
 ('each', 'each', 'each'),
 ('other', 'other', 'other'),
 ('that', 'that', 'that'),
 ('much', 'much', 'much'),
 ('.', '.', '.')]

In [146]:
light_result = light_model.fullAnnotate("John and Peter are brothers. However they don't support each other that much.")

In [147]:
light_result

Out[130]: [{'document': [Annotation(document, 0, 76, John and Peter are brothers. However they don't support each other that much., {})],
 'token': [Annotation(token, 0, 3, John, {'sentence': '0'}),
 Annotation(token, 5, 7, and, {'sentence': '0'}),
 Annotation(token, 9, 13, Peter, {'sentence': '0'}),
 Annotation(token, 15, 17, are, {'sentence': '0'}),
 Annotation(token, 19, 26, brothers, {'sentence': '0'}),
 Annotation(token, 27, 27, ., {'sentence': '0'}),
 Annotation(token, 29, 35, However, {'sentence': '0'}),
 Annotation(token, 37, 40, they, {'sentence': '0'}),
 Annotation(token, 42, 46, don't, {'sentence': '0'}),
 Annotation(token, 48, 54, support, {'sentence': '0'}),
 Annotation(token, 56, 59, each, {'sentence': '0'}),
 Annotation(token, 61, 65, other, {'sentence': '0'}),
 Annotation(token, 67, 70, that, {'sentence': '0'}),
 Annotation(token, 72, 75, much, {'sentence': '0'}),
 Annotation(token, 76, 76, ., {'sentence': '0'})],
 'stem': [Annotation(token, 0, 3, john, {'sentence': '0'}),
 Annotation(token, 5, 7, and, {'sentence': '0'}),
 Annotation(token, 9, 13, peter, {'sentence': '0'}),
 Annotation(token, 15, 17, ar, {'sentence': '0'}),
 Annotation(token, 19, 26, brother, {'sentence': '0'}),
 Annotation(token, 27, 27, ., {'sentence': '0'}),
 Annotation(token, 29, 35, howev, {'sentence': '0'}),
 Annotation(token, 37, 40, thei, {'sentence': '0'}),
 Annotation(token, 42, 46, don't, {'sentence': '0'}),
 Annotation(token, 48, 54, support, {'sentence': '0'}),
 Annotation(token, 56, 59, each, {'sentence': '0'}),
 Annotation(token, 61, 65, other, {'sentence': '0'}),
 Annotation(token, 67, 70, that, {'sentence': '0'}),
 Annotation(token, 72, 75, much, {'sentence': '0'}),
 Annotation(token, 76, 76, ., {'sentence': '0'})],
 'lemma': [Annotation(token, 0, 3, John, {'sentence': '0'}),
 Annotation(token, 5, 7, and, {'sentence': '0'}),
 Annotation(token, 9, 13, Peter, {'sentence': '0'}),
 Annotation(token, 15, 17, be, {'sentence': '0'}),
 Annotation(token, 19, 26, brother, {'sentence': '0'}),
 Annotation(token, 27, 27, ., {'sentence': '0'}),
 Annotation(token, 29, 35, However, {'sentence': '0'}),
 Annotation(token, 37, 40, they, {'sentence': '0'}),
 Annotation(token, 42, 46, don't, {'sentence': '0'}),
 Annotation(token, 48, 54, support, {'sentence': '0'}),
 Annotation(token, 56, 59, each, {'sentence': '0'}),
 Annotation(token, 61, 65, other, {'sentence': '0'}),
 Annotation(token, 67, 70, that, {'sentence': '0'}),
 Annotation(token, 72, 75, much, {'sentence': '0'}),
 Annotation(token, 76, 76, ., {'sentence': '0'})]}]

In [148]:
text_list= ["How did serfdom develop in and then leave Russia ?",
"There will be some exciting breakthroughs in NLP this year."]

light_model.annotate(text_list)

Out[131]: [{'document': ['How did serfdom develop in and then leave Russia ?'],
 'token': ['How',
 'did',
 'serfdom',
 'develop',
 'in',
 'and',
 'then',
 'leave',
 'Russia',
 '?'],
 'stem': ['how',
 'did',
 'serfdom',
 'develop',
 'in',
 'and',
 'then',
 'leav',
 'russia',
 '?'],
 'lemma': ['How',
 'do',
 'serfdom',
 'develop',
 'in',
 'and',
 'then',
 'leave',
 'Russia',
 '?']},
 {'document': ['There will be some exciting breakthroughs in NLP this year.'],
 'token': ['There',
 'will',
 'be',
 'some',
 'exciting',
 'breakthroughs',
 'in',
 'NLP',
 'this',
 'year',
 '.'],
 'stem': ['there',
 'will',
 'be',
 'some',
 'excit',
 'breakthrough',
 'in',
 'nlp',
 'thi',
 'year',
 '.'],
 'lemma': ['There',
 'will',
 'be',
 'some',
 'exciting',
 'breakthrough',
 'in',
 'NLP',
 'this',
 'year',
 '.']}]

In [149]:
## important note: When you use Finisher in your pipeline, regardless of setting cleanAnnotations to False or True, LigtPipeline will only return the finished columns.